<a href="https://colab.research.google.com/github/AlexeySto/A-B_test_DZ_seminar_8/blob/main/A_B_test_DZ_seminar_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Загрузка данных
import pandas as pd


data = pd.read_csv('/content/ab_stats.csv')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        23652 non-null  float64
 1   num_purchases  23652 non-null  int64  
 2   purchase       23652 non-null  int64  
 3   ab_group       23652 non-null  object 
 4   av_site visit  23652 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 924.0+ KB


,revenue,num_purchases,purchase,av_site visit
count,23652.000000,23652.000000,23652.000000,23652.000000
mean,0.324689,0.043590,0.020717,7.013112
std,9.557730,1.079403,0.142438,3.154584
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.173787
50%,0.000000,0.000000,0.000000,7.007936
75%,0.000000,0.000000,0.000000,8.864119
max,1303.609284,152.000000,1.000000,22.446822


In [43]:
# Очистка данных
# Удаление дубликатов
data.drop_duplicates(inplace=True)

# Обработка пропущенных значений, если таковые есть
data.dropna(inplace=True)

# Проверка на выбросы (если необходимо)
data.describe()

#Удаляем отрицательные значения
data = data.rename(columns= {'av_site visit': 'av_site_visit'})
data = data.query('av_site_visit >= 0')

data.info()
data.describe()
data.ab_group.value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 19534 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        19534 non-null  float64
 1   num_purchases  19534 non-null  int64  
 2   purchase       19534 non-null  int64  
 3   ab_group       19534 non-null  object 
 4   av_site_visit  19534 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 915.7+ KB


ab_group
A    9985
B    9549
Name: count, dtype: int64

In [44]:
# Проверка нормальности распределения данных из столбца 'revenue'
from scipy.stats import shapiro


statistic, p_value = shapiro(data['revenue'])

# Вывод результатов
if p_value > 0.05:
    print("Распределение данных 'revenue' является нормальным (p-value = {:.4f})".format(p_value))
else:
    print("Распределение данных 'revenue' не является нормальным (p-value = {:.4f})".format(p_value))


Распределение данных 'revenue' не является нормальным (p-value = 0.0000)


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



In [57]:
# Расчет мощности теста
from statsmodels.stats.power import TTestIndPower


effect_size = 0.5 #  эффект размера, который вы ожидаете увидеть в тесте.
# Это разница в средних значениях между двумя группами,
# нормированная на стандартное отклонение (обычно используется значение от 0.2 до 0.5).

alpha = 0.05 # уровень значимости, обычно 0.05.
nobs1 = data[data['ab_group'] == 'A']['revenue'].count()  # количество наблюдений в первой группе.
ratio = 1.0  # соотношение размеров выборок в группах (обычно 1.0).

power_analysis = TTestIndPower()
power = power_analysis.solve_power(effect_size=effect_size, alpha=alpha, nobs1=nobs1, ratio=ratio, power = None)

print(f"Мощность теста составляет: {power}")
if power < 0.8:
    print("Тест не достаточно мощный")
else:
    print("Тест соответствует ожидаемой мощности.")

Мощность теста составляет: 1.0
Тест соответствует ожидаемой мощности.


In [59]:
# Анализ данных
# Так как данные распределены не нормально используем критерий Мана-Уитни
from scipy.stats import mannwhitneyu


mw_stats = mannwhitneyu(x = data[data['ab_group'] == 'A']['revenue'], y = data[data['ab_group'] == 'A']['revenue'])
print(mw_stats)

alpha = 0.05
if mw_stats.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. Существует статистически значимое различие между выборками.")
else:
    print("Нет оснований отвергать нулевую гипотезу. Различия между выборками статистически не значимы.")

MannwhitneyuResult(statistic=49850112.5, pvalue=1.0)
Нет оснований отвергать нулевую гипотезу. Различия между выборками статистически не значимы.


Исходя из этих результатов, можно дать следующие рекомендации менеджеру:

1. На основе текущих данных нет оснований считать, что внедрение изменений, проводимых в рамках теста, влияет на целевую метрику (доход).

2. Рекомендуется проанализировать причины отсутствия различий между группами. Может быть, необходимо пересмотреть методику проведения теста, качество данных или проверить гипотезы, лежащие в основе теста.

3. Дополнительные исследования или изменения в дизайне эксперимента могут потребоваться для выявления факторов, влияющих на доход и эффективность проводимых изменений.

4. Важно быть внимательным к дальнейшим шагам, основываясь на анализе результатов A/B теста и возможно пересмотреть стратегию на основе новых данных и выводов.

В целом, рекомендую обсудить с командой проекта и аналитиками следующие шаги и возможные улучшения, чтобы получить более информативные результаты из будущих экспериментов.